In [1]:
import os
import openai
from openai.error import RateLimitError, APIConnectionError, InvalidRequestError
import pandas as pd
import time

In [ ]:
#pip install openai==0.28

#### GPT-4 configuration

#### GPT-3.5 configuration

In [2]:
azure_endpoint = "https://rumi-v0.openai.azure.com/"
api_key = "bb40601a473b4e25ad9c3975bd3c47ce"
api_version = "2023-07-01-preview"
openai.api_type = "azure"
openai.api_base = azure_endpoint
openai.api_key = api_key
openai.api_version = api_version

In [3]:
def load_prompt(review):
    # Read the prompt
    with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/prompt_for_decomposing_review_restaurants.txt', 'r') as file:
    #with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/prompt_for_decomposing_review_hairsalons.txt', 'r') as file:
        prompt = file.read()
        
        # Read the few-shot examples
        with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/few_shot_for_decomposing_review_restaurants.txt', 'r') as fewshot_file:
        #with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/few_shot_for_decomposing_review_hairsalons.txt', 'r') as fewshot_file:
            few_shot_examples = fewshot_file.read()
        
        prompt = prompt.format(review=review, few_shot_examples=few_shot_examples)
        #print(prompt)

    return prompt

In [4]:
reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/restaurants/test/rcb_using_decomposed_review_classification_v3_gpt35/restaurant_reviews_F.xlsx'
#reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hotels/dev/dev_w_ids.xlsx'
#reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hair_salons/dev/dev_w_ids.xlsx'
reviews_w_AtA_df = pd.read_excel(reviews_w_AtA)
reviews_w_AtA_df.head(10)

,name,business_id,review,decomposed_review,true_strong,true_weak,true_other,true_strong_weak,abs_true_strong,abs_true_weak,abs_true_other,abs_true_strong_weak_old,abs_true_strong_weak
0,Cochon Butcher,gTC8IQ_i8zXytWSly3Ttvg,"""So good that we came here twice and bought a ...",The restaurant is so good that the reviewer vi...,t-shirt,"Star wars memorabilia, Yoda",NaN,"t-shirt, Star wars memorabilia, Yoda",A customer bought a t-shirt.,The restaurant had Star wars memorabilia dress...,NaN,A customer bought a t-shirt.\nThe restaurant h...,A customer bought a t-shirt from the restauran...
1,The Pour House,RfO-dnZM_KCIPELt6q0dWg,2.5 Stars - Tolerable but Meh overall. Service...,The restaurant has a sports bar atmosphere. Th...,giant jenga,NaN,NaN,giant jenga,The restaurant has a giant jenga.,NaN,NaN,The restaurant has a giant jenga.,The restaurant has a giant jenga.
2,Capt'n Jack's Waterfront Grille,E7PBGs3fiZGyjcriapKcfQ,"A bit high priced, and the food is overall goo...","The restaurant is a bit high priced, and the f...","water, boaters, dock, your pets",NaN,parakeet,"water, boaters, dock, your pets",The restaurant is placed on the water.\nBoater...,NaN,A customer brought his parakeet on his shoulder.,The restaurant is placed on the water.\nBoater...,The restaurant is placed on the water.\nBoater...
3,Sol,OZnQZ4sZRwd7L1jMc7pv2w,A bit of self care and indulgence for when you...,The restaurant provides a bit of self care and...,"lounge area, symbolism, custom associations",NaN,NaN,"lounge area, symbolism, custom associations",The restaurant has a lounge area behind the ba...,NaN,NaN,The restaurant has a lounge area behind the ba...,The restaurant has a lounge area behind the ba...
4,Peshwa,UVsD8jyo-9J0OUdoSKI0jQ,A group of work friends and I stumbled upon up...,"The restaurant, Upper Deck, was stumbled upon ...","life size beer pong, pool table, arcade games,...",NaN,NaN,"life size beer pong, pool table, arcade games,...",The restaurant offers life size beer pong at t...,NaN,NaN,The restaurant offers life size beer pong at t...,The restaurant offers life size beer pong at t...
5,Bridgets Steakhouse,V5tCeElfEgLsWN78eJgKhA,After being in the restaurant business my enti...,The restaurant is a hidden gem in the restaura...,NaN,"their sister restaurant, lively fun band",NaN,"their sister restaurant, lively fun band",They have their sister restaurant next door of...,NaN,NaN,They have their sister restaurant next door of...,The restaurant has its sister restaurant next ...
6,Restaurant des Familles,AM7gPGpRbEjEkjmsmQXb6w,Amazing!!! Go for Sunday brunch. 5 course meal...,The restaurant offers a 5-course meal for roug...,"gators, airboat ride",NaN,NaN,"gators, airboat ride",The restaurant offers a great view of gators i...,NaN,NaN,The restaurant offers a great view of gators i...,The restaurant offers a great view of gators i...
7,Pat O'Brien’s,6Ty-KKWq6hLZYW8DWEHYvg,Best piano bar I've been to. In fact the rest ...,The restaurant is the best piano bar I've been...,best piano bar,NaN,NaN,"best piano bar, musicians",The restaurant is a piano bar.,NaN,NaN,The restaurant is a piano bar.,The restaurant is a piano bar.
8,Tarpon Tavern,Rxf7eEfub8LC27P28ObgsA,Came for the BOGO fish and chips last Tuesday ...,The restaurant offers a BOGO fish and chips de...,NaN,English Pub feel,NaN,English Pub feel,NaN,The restaurant has an old world sort of Englis...,NaN,The restaurant has an old world sort of Englis...,The restaurant has an old world sort of Englis...
9,Lagerhaus Brewery & Grill,yYnp7uLBt-PPyG2elubezg,Came here with a fellow Masonic brother for a ...,The restaurant offers a good dinner experience...,"amazing tour de bier, history lesson, cultural...",NaN,NaN,"amazing tour de bier, history lesson, cultural...",A group of customers was able to meet the brew...,NaN,NaN,A group of customers was able to meet the brew...,A group of customers was able to meet the brew...


### GPT-4

### GPT-3.5

In [5]:
data_to_process = []

for review_index, review_row in reviews_w_AtA_df.iterrows():
    review = review_row['review']
    name = review_row['name']
    business_id = review_row['business_id']
    data_to_process.append((name, business_id, review))
    
results = []

for name, business_id, review in data_to_process:
    try:
        
        prompt = load_prompt(review)
        
        messages = [
              {"role": "system", "content": "You are a helpful AI assistant"},
              {"role": "user", "content": prompt}
        ]
        
        formatted_messages = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
        print("Messages being sent to GPT:\n", formatted_messages)

        # Use OpenAI's chat completion endpoint 
        response = openai.ChatCompletion.create(
            messages=messages,
            engine="gpt-35-turbo",
            temperature=0,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=500
        )
        
        # Append the result to the list
        result_content = response['choices'][0]['message']['content']
        results.append((name, business_id, review, result_content))
        print(results)


    except openai.error.InvalidRequestError as e:
        print(f"Skipping due to error: {e}")
    except RateLimitError as e:
        print("Rate limit exceeded. Retrying in 150 seconds...")
        time.sleep(150)
    except APIConnectionError as e:
        print("APIConnectionError: Failed to connect to the API. Please check your network connection and API configuration:", e)
    except Exception as e:
        print("An unexpected error occurred:", e)

Messages being sent to GPT:
 system: You are a helpful AI assistant
user: You will be given a restaurant review. Read the provided review thoroughly and generate a list of sentences that only talk about features or properties of the restaurant. You can remove redundant statements and opinions as long as the generated list of sentences is comprehensive in that it should include all the restaurant features that were mentioned in the original review. The generated sentences should stay as close as possible to the sentences in the original review.

Here are some examples below:

Example 1:

A bit of self care and indulgence for when you want to relax and de-stress and if you don't want to go to the bar but have another option. Relax and enjoy. There's a lounge area behind. So you can take your drink in the lounge and kick back. Caffeinated choices are available. If you like your tastebuds teased in a different way, this is a treat. Perfect for yogis and if you like incense. (there's no inc

[('Cochon Butcher', 'gTC8IQ_i8zXytWSly3Ttvg', '"So good that we came here twice and bought a t-shirt" This was probably one of the top 3 restaurants for my wife and I. We visited twice and can understand the hype. From the service to the food: amazing! Ambience: Hipster? Small amount of seating. Fits 40ish people. Eclectic. Star wars memorabilia dressed with random costumes and items. Yoda riding a unicorn. That sums it up. Clean and organized.Pricing: Quality at the right price. They are not throwing huge portions at you, but you are not getting ripped off. You\'re getting what you pay for. Reminder: they take the time to craft your food which means love and care was added. Worth it. Food: You order at the counter and hopefully you can find a seat after. Thankfully the selection is enough where you don\'t feel overwhelmed. This adds to the wait in line. Get what you know you love. You won\'t be disappointed. You don\'t have to follow what people suggest unless you guys have similar pa

[('Cochon Butcher', 'gTC8IQ_i8zXytWSly3Ttvg', '"So good that we came here twice and bought a t-shirt" This was probably one of the top 3 restaurants for my wife and I. We visited twice and can understand the hype. From the service to the food: amazing! Ambience: Hipster? Small amount of seating. Fits 40ish people. Eclectic. Star wars memorabilia dressed with random costumes and items. Yoda riding a unicorn. That sums it up. Clean and organized.Pricing: Quality at the right price. They are not throwing huge portions at you, but you are not getting ripped off. You\'re getting what you pay for. Reminder: they take the time to craft your food which means love and care was added. Worth it. Food: You order at the counter and hopefully you can find a seat after. Thankfully the selection is enough where you don\'t feel overwhelmed. This adds to the wait in line. Get what you know you love. You won\'t be disappointed. You don\'t have to follow what people suggest unless you guys have similar pa

[('Cochon Butcher', 'gTC8IQ_i8zXytWSly3Ttvg', '"So good that we came here twice and bought a t-shirt" This was probably one of the top 3 restaurants for my wife and I. We visited twice and can understand the hype. From the service to the food: amazing! Ambience: Hipster? Small amount of seating. Fits 40ish people. Eclectic. Star wars memorabilia dressed with random costumes and items. Yoda riding a unicorn. That sums it up. Clean and organized.Pricing: Quality at the right price. They are not throwing huge portions at you, but you are not getting ripped off. You\'re getting what you pay for. Reminder: they take the time to craft your food which means love and care was added. Worth it. Food: You order at the counter and hopefully you can find a seat after. Thankfully the selection is enough where you don\'t feel overwhelmed. This adds to the wait in line. Get what you know you love. You won\'t be disappointed. You don\'t have to follow what people suggest unless you guys have similar pa

[('Cochon Butcher', 'gTC8IQ_i8zXytWSly3Ttvg', '"So good that we came here twice and bought a t-shirt" This was probably one of the top 3 restaurants for my wife and I. We visited twice and can understand the hype. From the service to the food: amazing! Ambience: Hipster? Small amount of seating. Fits 40ish people. Eclectic. Star wars memorabilia dressed with random costumes and items. Yoda riding a unicorn. That sums it up. Clean and organized.Pricing: Quality at the right price. They are not throwing huge portions at you, but you are not getting ripped off. You\'re getting what you pay for. Reminder: they take the time to craft your food which means love and care was added. Worth it. Food: You order at the counter and hopefully you can find a seat after. Thankfully the selection is enough where you don\'t feel overwhelmed. This adds to the wait in line. Get what you know you love. You won\'t be disappointed. You don\'t have to follow what people suggest unless you guys have similar pa

[('Cochon Butcher', 'gTC8IQ_i8zXytWSly3Ttvg', '"So good that we came here twice and bought a t-shirt" This was probably one of the top 3 restaurants for my wife and I. We visited twice and can understand the hype. From the service to the food: amazing! Ambience: Hipster? Small amount of seating. Fits 40ish people. Eclectic. Star wars memorabilia dressed with random costumes and items. Yoda riding a unicorn. That sums it up. Clean and organized.Pricing: Quality at the right price. They are not throwing huge portions at you, but you are not getting ripped off. You\'re getting what you pay for. Reminder: they take the time to craft your food which means love and care was added. Worth it. Food: You order at the counter and hopefully you can find a seat after. Thankfully the selection is enough where you don\'t feel overwhelmed. This adds to the wait in line. Get what you know you love. You won\'t be disappointed. You don\'t have to follow what people suggest unless you guys have similar pa

KeyboardInterrupt: 

In [ ]:
data_to_process = []

for review_index, review_row in reviews_w_AtA_df.iterrows():
    review = review_row['review']
    name = review_row['name']
    business_id = review_row['business_id']
    data_to_process.append((name, business_id, review))
    
results = []

for name, business_id, review in data_to_process:
    #try:
        
    prompt = load_prompt(review)

    messages = [
          {"role": "system", "content": "You are a helpful AI assistant"},
          {"role": "user", "content": prompt}
    ]

    formatted_messages = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
    print("Messages being sent to GPT:\n", formatted_messages)

    # Use OpenAI's chat completion endpoint 
    response = openai.ChatCompletion.create(
        messages=messages,
        engine="gpt-4o-tonmoy",
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=500
    )

    # Append the result to the list
    result_content = response['choices'][0]['message']['content']
    results.append((name, business_id, review, result_content))
    print(results)


    '''except openai.error.InvalidRequestError as e:
        print(f"Skipping due to error: {e}")
    except RateLimitError as e:
        print("Rate limit exceeded. Retrying in 150 seconds...")
        time.sleep(150)
    except APIConnectionError as e:
        print("APIConnectionError: Failed to connect to the API. Please check your network connection and API configuration:", e)
    except Exception as e:
        print("An unexpected error occurred:", e)'''

In [ ]:
corrected_results = []
for r in results:
    if len(r) == 4:
        name, business_id, review, decomposed_review = r
        corrected_results.append((name, business_id, review, decomposed_review))
    else:
        print(f"Skipping incorrect entry: {r}")

# Create a DataFrame
results_df = pd.DataFrame(corrected_results, columns=['name', 'business_id', 'review', 'decomposed_review'])

# Define output file path
output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/restaurants/test/rcb_using_decomposed_review_classification_v3_gpt35/train_test_w_ids_decomposed.xlsx'
#output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hotels/dev/dev_w_ids_decomposed.xlsx'
#output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hair_salons/dev/dev_w_ids_decomposed.xlsx'

# Save the DataFrame to an Excel file
results_df.to_excel(output_file, index=False)

print(f"Data successfully saved to {output_file}")